# Работа с таблицами в библиотеке Pandas ч. 2.2
* Преобразование датафреймов pandas: добавление, удаление строк и столбцов в таблице, фильтрация строк по условиям. Группировка и агрегирование данных. Объединение таблиц.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as sps

## 1. Простые операции

Сгенерируем случайные числа и представим их в виде `DataFrame`.

In [3]:
df = pd.DataFrame(sps.norm.rvs(size=(10, 4)), 
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0.252994,0.990995,0.102464,0.420442
1,0.151090,-0.645166,0.589453,-0.144247
2,-0.101071,-0.046478,-0.890034,-0.084252
3,1.067270,-0.796233,0.404484,-0.257340
4,-1.023416,-0.709547,0.530971,-0.008997
5,0.081104,-0.628490,1.186595,0.388769
6,-0.277434,0.264491,1.116479,-1.132930
7,-0.226425,-0.289982,-0.294458,0.527639
8,-0.254820,-0.156553,-0.575165,-0.354650
9,0.416030,0.462873,0.253105,0.218463


Выведем описательные статистики по столбцам &mdash; количество значений, среднее, стандартное отклонение (корень из дисперсии), минимум, квантили, максимум.

In [4]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.008532,-0.155409,0.242389,-0.042710
std,0.543200,0.585396,0.678580,0.487547
min,-1.023416,-0.796233,-0.890034,-1.132930
25%,-0.247721,-0.640997,-0.195228,-0.229067
50%,-0.009984,-0.223267,0.328795,-0.046624
75%,0.227518,0.186748,0.574833,0.346192
max,1.067270,0.990995,1.186595,0.527639


Среднее по столбцам

In [5]:
df.mean()

A    0.008532
B   -0.155409
C    0.242389
D   -0.042710
dtype: float64

Оценка матрицы корреляций значений в столбцах

In [6]:
df.corr()

,A,B,C,D
A,1.000000,0.077337,0.053527,0.100239
B,0.077337,1.000000,-0.200658,0.046015
C,0.053527,-0.200658,1.000000,-0.225680
D,0.100239,0.046015,-0.225680,1.000000


Применение функции к данным.
Для примера посчитаем разброс значений &mdash; разница максимума и минимума.

In [7]:
df.apply(lambda x: x.max() - x.min())

A    2.090686
B    1.787228
C    2.076629
D    1.660569
dtype: float64

## 2. Объединение таблиц

### 2.2 Функция `pd.concat`

Соединение таблиц вдоль выбранной оси

`pd.concat(objs, axis=0, join='outer', ignore_index=False, copy=True, ...)`

* `objs` &mdash; объединяемые таблицы;
* `axis` : {`0` или `'index'`, `1` или `'columns'`} &mdash; ось индексов или ось колонок, иными словами соединение по вертикали или по горизонтали;
* `join` : {`'inner'`, `'outer'`} &mdash; тип объединения &mdash; пересечение или объединение индексов/колонок;
* `ignore_index` &mdash; сохранить индексы или определить и как $0, ..., n-1$;
* `copy` &mdash; копировать данные или нет.

------------

Простой пример соединения таблиц:

In [13]:
df

,A,B,C,D
0,0.252994,0.990995,0.102464,0.420442
1,0.151090,-0.645166,0.589453,-0.144247
2,-0.101071,-0.046478,-0.890034,-0.084252
3,1.067270,-0.796233,0.404484,-0.257340
4,-1.023416,-0.709547,0.530971,-0.008997
5,0.081104,-0.628490,1.186595,0.388769
6,-0.277434,0.264491,1.116479,-1.132930
7,-0.226425,-0.289982,-0.294458,0.527639
8,-0.254820,-0.156553,-0.575165,-0.354650
9,0.416030,0.462873,0.253105,0.218463


In [20]:
pd.concat([df[:3], df[:3]])

,A,B,C,D
0,0.252994,0.990995,0.102464,0.420442
1,0.151090,-0.645166,0.589453,-0.144247
2,-0.101071,-0.046478,-0.890034,-0.084252
0,0.252994,0.990995,0.102464,0.420442
1,0.151090,-0.645166,0.589453,-0.144247
2,-0.101071,-0.046478,-0.890034,-0.084252


### 2.3 Функции `pd.merge` и `df.join`

Слияние таблиц по вертикали путем выполнения операций слияния баз данных в стиле SQL.

`pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, suffixes=('_x', '_y'), ...)`

* `left` и `right` &mdash; объединяемые таблицы.
* `how` &mdash; тип объединения:
    * `left` &mdash; только по ключам из левой таблицы == *SQL left outer join*;
    * `right` &mdash; только по ключам из правой таблицы == *SQL right outer join*;
    * `outer` &mdash; по объединению ключей == *SQL full outer join*;
    * `inner` &mdash; по пересечению ключей == *SQL inner join*.
* `on` &mdash; имя (или имена) колонок, по которым будет производиться объединение (т.е. ключи). Если их несколько, то нужно передать список имен. Имена колонок в таблице должны совпадать.
* `left_on` и `right_on` &mdash; аналогично `on` для случая, когда в таблицах различаются имена колонок, соответствующие ключам.
* `left_index` и `right_index` &mdash; использовать ли индексы в качестве ключей.
* `suffixes` &mdash; суффиксы, которые будут добавлены к тем колонкам, имена которых повторяются.

*Пример.* Опция `how=left, left_on='A', right_on='B'` соответствует взятию всех строк из таблицы `left`, а из таблицы `right` берутся те строки, в которых значения в колонке `A` таблицы `left` совпадает со значением колонки `B` таблицы `right`. Если в одной из таблиц таких значений несколько, то строки другой таблицы дублируются. Если в таблице `right` каких-то значений нет, то в результирующей таблице будут пропуски.

`df.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)`

* `df` &mdash; основная таблица. В качестве ключей используется индекс.
* `other` &mdash; другая таблица.
* `on` &mdash; колонка(-и) в `other`, соответствующая ключам, по ним происходит объедиенение. Если `None`, то используется индекс.
* `how` &mdash; тип объединения (см. `pd.merge`).
* `lsuffix` и `rsuffix` &mdash; суффиксы, которые будут добавлены к тем колонкам, имена которых повторяются.

------------------

##### Пример 1.

В обеих таблицах ключи повторяются

In [9]:
left = pd.DataFrame({'key': ['A', 'A'], 
                     'lval': [1, 2]})
right = pd.DataFrame({'key': ['A', 'A'], 
                      'rval': [4, 5]})

In [10]:
left

,key,lval
0,A,1
1,A,2


In [11]:
right

,key,rval
0,A,4
1,A,5


В результате объединения получаем 4 строки &mdash; для каждой строки из левой таблице есть две строки из правой таблицы с таким же ключом.

In [12]:
pd.merge(left, right, on='key')

,key,lval,rval
0,A,1,4
1,A,1,5
2,A,2,4
3,A,2,5


###### Пример 2.

В таблицах ключи не повторяются

In [13]:
left = pd.DataFrame({'key': ['A', 'B'], 
                     'lval': [1, 2]})
right = pd.DataFrame({'key': ['A', 'B'], 
                      'rval': [4, 5]})

In [14]:
left

,key,lval
0,A,1
1,B,2


In [15]:
right

,key,rval
0,A,4
1,B,5


В результате объединения получаем 2 строки &mdash; для каждой строки из левой таблице есть только одна строка из правой таблицы с таким же ключом.

In [16]:
pd.merge(left, right, on='key')

,key,lval,rval
0,A,1,4
1,B,2,5


###### Пример 3.

Посмотрим на различные типы объединения. Сооздадим и напечатаем две таблицы.

In [17]:
left = pd.DataFrame({'lkey': ['A', 'B', 'C', 'A'], 
                     'value': range(4)})
right = pd.DataFrame({'rkey': ['A', 'B', 'D', 'B'], 
                      'value': range(4, 8)})

In [18]:
left

,lkey,value
0,A,0
1,B,1
2,C,2
3,A,3


In [19]:
right

,rkey,value
0,A,4
1,B,5
2,D,6
3,B,7


**Внешнее слияние** &mdash; используются ключи из объединения списков ключей. Иначе говоря, используются ключи, которые есть хотя бы в одной из таблиц. Если в другой таблице таких ключей нет, то ставятся пропуски.

In [20]:
pd.merge(left, right, 
         left_on='lkey', right_on='rkey', how='outer')

,lkey,value_x,rkey,value_y
0,A,0.0,A,4.0
1,A,3.0,A,4.0
2,B,1.0,B,5.0
3,B,1.0,B,7.0
4,C,2.0,NaN,NaN
5,NaN,NaN,D,6.0


**Внутреннее слияние** &mdash; используются ключи из пересечения списков ключей. Иначе говоря, используются ключи, которые присутствуют в обеих таблицах.

In [21]:
pd.merge(left, right, 
         left_on='lkey', right_on='rkey', how='inner')

,lkey,value_x,rkey,value_y
0,A,0,A,4
1,A,3,A,4
2,B,1,B,5
3,B,1,B,7


**Объединение по ключам левой таблицы.** Не используются ключи, которые есть в правой таблицы, но которых нет в левой. Если в правой таблице каких-то ключей нет, то ставятся пропуски.

In [22]:
pd.merge(left, right, 
         left_on='lkey', right_on='rkey', how='left')

,lkey,value_x,rkey,value_y
0,A,0,A,4.0
1,B,1,B,5.0
2,B,1,B,7.0
3,C,2,NaN,NaN
4,A,3,A,4.0


**Объединение по ключам правой таблицы.** Не используются ключи, которые есть в левой таблицы, но которых нет в правой. Если в левой таблице каких-то ключей нет, то ставятся пропуски.

In [23]:
pd.merge(left, right, 
         left_on='lkey', right_on='rkey', how='right')

,lkey,value_x,rkey,value_y
0,A,0.0,A,4
1,A,3.0,A,4
2,B,1.0,B,5
3,NaN,NaN,D,6
4,B,1.0,B,7


Выполним внтуреннее объединение и установим ключ качестве индекса

In [24]:
pd.merge(left, right, 
         left_on='lkey', right_on='rkey', how='inner') \
        .set_index('lkey')[['value_x', 'value_y']]

,value_x,value_y
lkey,,
A,0,4
A,3,4
B,1,5
B,1,7


Ту же операцию можно выполнить с помощью `join`

In [25]:
left.set_index('lkey') \
    .join(right.set_index('rkey'), rsuffix='_r', how='inner')

,value,value_r
A,0,4
A,3,4
B,1,5
B,1,7


### 3. Группировка

Часто на практике необходимо вычислять среднее по каким-либо категориям или группам в данных. Группа может определяться, например, столбцом в таблице, у которого не так много значений. Мы хотели бы для каждого такого значения посчитать среднее значение другой колонки данных в этой группе.

Этапы группировки данных:

* разбиение данных на группы по некоторым критериям;
* применение функции отдельно к каждой группе;
* комбинирование результата в структуру данных.

Группировка выполняется функцией

`df.groupby(by=None, axis=0, level=None, sort=True, ...)`

* `df` &mdash; таблица, данные которой должны быть сгруппированы;
* `by` &mdash; задает принцип группировки. Чаще всего это имя столбца, по которому нужно сгруппировать. Может так же быть функцией;
* `axis` &mdash; ось (0 = группировать строки, 1 = группировать столбцы);
* `level` &mdash; если ось представлена мультииндексом, то указывает на уровень мультииндекса;
* `sort` &mdash; сортировка результата по индексу.

Результатом группировки является объект, состоящий из пар (имя группы, подтаблица). Имя группы соответствует значению, по которому произведена группировка. К объекту-результату группировки применимы, например, следующие операции:

* `for name, group in groupped: ... ` &mdash; цикл по группам;
* `get_group(name)` &mdash; получить таблицу, соответствующую группе с именем `name`;
* `groups` &mdash; получить все группы в виде словаря имя-подтаблица;
* `count()` &mdash; количество значений в группах, исключая пропуски;
* `size()` &mdash; размер групп;
* `sum()`, `max()`, `min()`;
* `mean()`, `median()`, `var()`, `std()`, `corr()`, `quantile(q)`;
* `describe()` &mdash; вывод описательных статистик;
* `aggregate(func)` &mdash; применение функции (или списка функций) `func` к группам.
---------

Создадим таблицу для примера

In [26]:
df = pd.DataFrame({
    'Животное' : ['Котик', 'Песик', 'Котик', 'Песик',
                  'Котик', 'Песик', 'Котик', 'Песик'],
    'Цвет шерсти' : ['белый', 'белый', 'коричневый', 'черный',
                     'коричневый', 'коричневый', 'белый', 'черный'],
    'Рост' : sps.gamma(a=12, scale=3).rvs(size=8),
    'Длина хвостика' : sps.gamma(a=10).rvs(size=8)
})

df

,Животное,Цвет шерсти,Рост,Длина хвостика
0,Котик,белый,28.275071,14.689840
1,Песик,белый,36.176099,11.767005
2,Котик,коричневый,38.305081,8.801276
3,Песик,черный,33.749101,13.343875
4,Котик,коричневый,76.971116,13.545224
5,Песик,коричневый,32.298801,6.480564
6,Котик,белый,20.512039,8.805352
7,Песик,черный,26.765435,9.756060


##### Пример 1.

Если все котики встанут друг на друга, то какой их суммарный рост? А у песиков? А какова суммарная длинна хвостиков у котиков и у песиков?

Группировка по одной колонке и последующее применение операции суммирования:

In [27]:
df.groupby('Животное').sum()

,Рост,Длина хвостика
Животное,,
Котик,164.063307,45.841693
Песик,128.989437,41.347503


Посчитаем описательные статистики для каждого животного

In [28]:
df.groupby('Животное').describe()

Рост                                                         \
         count       mean        std        min        25%        50%   
Животное                                                                
Котик      4.0  41.015827  25.052366  20.512039  26.334313  33.290076   
Песик      4.0  32.247359   3.989334  26.765435  30.915460  33.023951   

                              Длина хвостика                                 \
               75%        max          count       mean       std       min   
Животное                                                                      
Котик     47.97159  76.971116            4.0  11.460423  3.103546  8.801276   
Песик     34.35585  36.176099            4.0  10.336876  2.960619  6.480564   

                                                     
               25%        50%        75%        max  
Животное                                             
Котик     8.804333  11.175288  13.831378  14.689840  
Песик     8.937186  10.761532  12.161222  13.343875

##### Пример 2.

Теперь предположим, что котики и песики встают только на представителей своего вида и своего цвета шерсти. Что тогда будет?

Группировка по двум колонкам и последующее применение операции суммирования

In [29]:
df.groupby(['Животное', 'Цвет шерсти']).sum()

Рост  Длина хвостика
Животное Цвет шерсти                            
Котик    белый         48.787110       23.495192
         коричневый   115.276198       22.346500
Песик    белый         36.176099       11.767005
         коричневый    32.298801        6.480564
         черный        60.514536       23.099935

Полученная таблица имеет *мультииндекс*

In [30]:
df.groupby(['Животное', 'Цвет шерсти']).sum().index

MultiIndex([('Котик',      'белый'),
            ('Котик', 'коричневый'),
            ('Песик',      'белый'),
            ('Песик', 'коричневый'),
            ('Песик',     'черный')],
           names=['Животное', 'Цвет шерсти'])

### 4. Cводные таблицы (Pivot table)

**Задача.** В медицинской клинике информацию о приемах записывают в таблицу со следующими полями:
* время приема,
* врач,
* пациент,
* поставленный диагноз,
* назначение,
* другие поля.

Требуется посчитать, сколько раз за предыдущий месяц каждый врач ставил какой-либо диагноз. Результаты представить в виде таблицы, в которой посчитать также суммы по строкам и столбцам, т.е. сколько врач сделал приемов за месяц и сколько раз конкретный диагноз поставлен всеми врачами.

Как решать?

**Способ 1** 
1. Группировка по врачам.
2. Для каждого врача группировка по диагнозам.
3. В каждой группе вычисление суммы.
4. Соединение в одну таблицу.
5. Вычисление суммы по столбцам и по строкам.

Можете прикинуть количество строк кода и время работы;

**Способ 2**
1. Создать пустую таблицу.
2. Циклом по всем записям исходной таблицы считать суммы.
3. Вычисление суммы по столбцам и по строкам.

И снова можете прикинуть количество строк кода и время работы;

**Способ 3**

Применить умную функцию из pandas, которая сделает все сама!

---------------


#### Функция `pd.pivot_table`

Эксель-подобные сводные таблицы

`pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`

* `data` &mdash; исходная таблица;
* `values` &mdash; аггригируемый столбец, его значения непосредственно определяют значения сводной таблицы;
* `index` &mdash; ключи для группировки, относятся к индексам сводной таблицы;
* `columns` &mdash; ключи для группировки, относятся к столбцам сводной таблицы;
* `aggfunc` &mdash; функция, которая будет применена к каждой группе значений `values`, сгруппированным по значениям `index` и `columns`. Значения этой функции и есть значения сводной таблицы. Если передается список функций, то сводная таблица имеет иерархические имена колонок, верхние значения которых &mdash; имена функций;
* `fill_value` &mdash; значения для замены пропусков;
* `dropna` &mdash; не включать столбцы, которые состоят только из `NaN`;
* `margins` &mdash; добавляет результирующий столбец/строку;
* `margins_name` &mdash; имя результирующего столбец/строку.

В примере выше:

`pd.pivot_table(df, index='врач', columns='диагноз', margins=True)`

-------------

#### 4.3 Примеры

Создадим таблицу для примера

In [31]:
df = pd.DataFrame({
    'Специальность' : ['Ветеринар', 'Ветеринар', 
                       'Психолог', 'Психолог'] * 6,
    'Врач' : ['Андрей', 'Сергей', 'Ирина'] * 8,
    'Диагноз' : ['Простуда', 'Простуда', 'Простуда', 
                 'Волнения', 'Волнения', 'Простуда'] * 4,
    'Доза' : sps.randint(low=1, high=6).rvs(size=24),
    'Продолжительность' : sps.randint(low=1, high=6).rvs(size=24)
})

df

,Специальность,Врач,Диагноз,Доза,Продолжительность
0,Ветеринар,Андрей,Простуда,3,5
1,Ветеринар,Сергей,Простуда,3,2
2,Психолог,Ирина,Простуда,4,5
3,Психолог,Андрей,Волнения,4,2
4,Ветеринар,Сергей,Волнения,1,2
5,Ветеринар,Ирина,Простуда,2,3
6,Психолог,Андрей,Простуда,5,3
7,Психолог,Сергей,Простуда,2,3
8,Ветеринар,Ирина,Простуда,4,2
9,Ветеринар,Андрей,Волнения,1,3


Простейший вариант сводной таблицы &mdash; среднее в группах, определяемых столбцом. Посчитаем средние по каждому врачу

In [34]:
pd.pivot_table(df, index=['Врач'])

,Доза,Продолжительность
Врач,,
Андрей,3.500,3.375
Ирина,3.250,3.875
Сергей,1.875,2.750


Посчитаем, сколько раз врач и в какой специальности ставил тот или иной диагноз

In [35]:
pd.pivot_table(df, 
               values='Доза', 
               index=['Специальность', 'Врач'],
               columns=['Диагноз'], 
               aggfunc=np.sum)

Диагноз               Волнения  Простуда
Специальность Врач                      
Ветеринар     Андрей       6.0       5.0
              Ирина        NaN      15.0
              Сергей       2.0       5.0
Психолог      Андрей       9.0       8.0
              Ирина        NaN      11.0
              Сергей       5.0       3.0

Добавим строчку, являющейся суммой столбцов, и столбец, являющийся суммой строк

In [36]:
pd.pivot_table(df, 
               values='Доза', 
               index=['Специальность', 'Врач'],
               columns=['Диагноз'],
               aggfunc=np.sum, 
               margins=True)

Диагноз               Волнения  Простуда  All
Специальность Врач                           
Ветеринар     Андрей       6.0       5.0   11
              Ирина        NaN      15.0   15
              Сергей       2.0       5.0    7
Психолог      Андрей       9.0       8.0   17
              Ирина        NaN      11.0   11
              Сергей       5.0       3.0    8
All                       22.0      47.0   69

Применим несколько функций и несколько столбцов со значениями

In [37]:
pd.pivot_table(df, 
               values=['Доза', 'Продолжительность'], 
               index=['Специальность', 'Врач'],
               columns=['Диагноз'], 
               aggfunc=[np.min, np.mean, np.max], 
               margins=True)

amin                                              \
                         Доза              Продолжительность                
Диагноз              Волнения Простуда All          Волнения Простуда All   
Специальность Врач                                                          
Ветеринар     Андрей      1.0      2.0   1               1.0      4.0   1   
              Ирина       NaN      2.0   2               NaN      2.0   2   
              Сергей      1.0      2.0   1               2.0      2.0   2   
Психолог      Андрей      4.0      3.0   3               2.0      3.0   2   
              Ирина       NaN      1.0   1               NaN      3.0   3   
              Сергей      1.0      1.0   1               1.0      3.0   1   
All                       1.0      1.0   1               1.0      2.0   1   

                         mean                                             \
                         Доза                 Продолжительность            
Диагноз              Волнения Простуда    All          Волнения Простуда   
Специальность Врач                                                         
Ветеринар     Андрей     3.00   2.5000  2.750             2.000   4.5000   
              Ирина       NaN   3.7500  3.750               NaN   3.5000   
              Сергей     1.00   2.5000  1.750             3.000   2.5000   
Психолог      Андрей     4.50   4.0000  4.250             3.500   3.5000   
              Ирина       NaN   2.7500  2.750               NaN   4.2500   
              Сергей     2.50   1.5000  2.000             2.000   3.5000   
All                      2.75   2.9375  2.875             2.625   3.6875   

                                   amax                                 \
                                   Доза              Продолжительность   
Диагноз                    All Волнения Простуда All          Волнения   
Специальность Врач                                                       
Ветеринар     Андрей  3.250000      5.0      3.0   5               3.0   
              Ирина   3.500000      NaN      5.0   5               NaN   
              Сергей  2.750000      1.0      3.0   3               4.0   
Психолог      Андрей  3.500000      5.0      5.0   5               5.0   
              Ирина   4.250000      NaN      4.0   4               NaN   
              Сергей  2.750000      4.0      2.0   4               3.0   
All                   3.333333      5.0      5.0   5               5.0   

                                   
                                   
Диагноз              Простуда All  
Специальность Врач                 
Ветеринар     Андрей      5.0   5  
              Ирина       5.0   5  
              Сергей      3.0   4  
Психолог      Андрей      4.0   5  
              Ирина       5.0   5  
              Сергей      4.0   4  
All                       5.0   5